In [24]:
try:
    import colab_requirements_install
    from google.colab import drive
    drive.mount('/content/drive')

    import os
    os.chdir('/content/drive/MyDrive/DL_Project/')
    tpu_cores = 8
    gpu_cores = None
    datapath = '??? TBD' # todo: work out what structure will be used in colab
    rootpath = '??? TBD'
    print("Initilaising in colab set-up...")
except(ModuleNotFoundError):
    print('Initialising non-colab set-up...')
    tpu_cores=None
    gpu_cores=None
    datapath = '../dataset/processed/'
    rootpath = '../'

from VGGnet import VGGnet
from dataloader import VoxDataloader
import pytorch_lightning.loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl
print('done.')

Initialising non-colab set-up...
done.


In [27]:
config = {
            # learning params
            'lr': 1e-3,
            'batch_size': 32,
            'early_stopping': True,
            'optimizer': 'SGD',
            'max_epochs': 100,
            'early_stopping': True,
            'patience': 5, # tolerance for early stopping

            #, regularization
            'dropout': 0.5,
            'L2': 0.1,
            'batch_norm': True,
            'gradient_clipping': 0.5,

}

In [28]:
logname = "opt{}_lr{}_reg{}_drop{}_bn{}_gc{}".format(*[config[s] for s in ['optimizer','lr','L2','dropout','batch_norm','gradient_clipping']])

dataloader = VoxDataloader(datapath, batch_size=config['batch_size'])
model = VGGnet(num_classes=dataloader.num_classes(), lr=config['lr'], batch_norm=config['batch_norm'])
tb_logger = pl_loggers.TensorBoardLogger(rootpath + './VGGlogs/', name=logname)
trainer = pl.Trainer(logger=tb_logger, max_epochs=config['max_epochs'], tpu_cores=tpu_cores, gpus=gpu_cores, log_every_n_steps=20,
                      callbacks=[EarlyStopping(
                          monitor='val_loss',
                          patience=config['patience'])] if config['early_stopping'] else None)

trainer.fit(model, dataloader)

KeyError: 'int_id'